# <center>Deepseek企业级Agent项目开发实战</center>

## <center>Part 5: Microsoft GraphRAG 多源数据索引构建方案</center>
### <center><font color=red>**关系型数据库与CSV文件格式优化策略**</font></center>

&emsp;&emsp;在前面的课程中我们依次介绍了`Microsoft GraphRAG` 在构建索引和问答检索两个流程的实现细节，在讲解原理的过程中主要是借助相对简单的`txt`文件来进行流程的演示，目的是在于帮助大家以尽可能低的门槛先跑通流程，并在这个过程中熟练掌握`Microsoft GraphRAG` 各个阶段的工作原理。

&emsp;&emsp;经历过第一个阶段的认知性学习后，我们从本节课开始将进入第二个阶段，即实战性学习阶段。

&emsp;&emsp;实际的落地需求场景中，往往需要处理的是更加复杂的数据，`.txt` 文件是最基础，同时也并不是特别常用的数据格式，更加常见的数据类型是`关系型数据库中的数据`、`CSV`、`JSON`、`PDF`、`DOC`等文件格式，每个文件格式都有其独特的数据结构和数据处理方式，比如：

- 关系型数据库中的数据有成百上千张表，数据量大且关联复杂；
- PDF文件中既包含文本信息，又包含图片、表格、公式等复杂结构；

&emsp;&emsp;同时，`Microsoft GraphRAG` 默认支持的文件类型仅有`.txt`、`.csv` 以及在刚刚发布的 `v2.0.1`版本新增的`.json` 文件格式，因此，如果要适配到实际的落地场景中，需要对以上几种类型做深度的集成及优化，才能更好的利用`Microsoft GraphRAG` 的优秀特性。所以接下来的课程，我们就针对`Microsoft GraphRAG` 最新发布的`v2.0.1`版本进行源码级别的二次开发，自定义接入`.pdf`、`.docx` 等文件类型的支持，并结合智能客服场景的数据，展开详细的优化策略和思路讲解。

&emsp;&emsp;本节课我们首先进行关系型数据库与`.csv` 文件的优化策略及在`Microsoft GraphRAG` 中集成后的使用方法。

# 1. 智能客服数据准备

&emsp;&emsp;智能客服领域自`2023`年起以大模型为基座的`RAG`与`Agent`技术架构开始出现以后，就一直是企业应用落地的热门方向。各行各业都在积极探索如何将大模型技术与业务场景相结合，以提升客户服务质量和效率。不过，如电商、金融、教育等不同行业的数据和场景差异其实非常大。比如电商领域，需要回答用户问到的订单状态、退货政策等问题，而金融的话可能更多是账户安全、投资产品的问题。

&emsp;&emsp;同时，智能客服场景使用的数据往往是企业内部非常私密的数据，是不太可能在公网上获取到相关的隐私数据的。因此，我们接下来给大家选择的数据集是来自`Kaggle`上的一个公开数据集 - `Northwind Traders`。其地址为：https://www.kaggle.com/datasets/jeetahirwar/northwind-traders

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211500581.png" width=60%></div>

&emsp;&emsp;`Northwind Traders` 是一个虚构的贸易公司数据集，最初由微软创建，用作 `Microsoft Access` 和其他数据库产品的示例数据库。它后来也被移植到了其他数据库平台，并在 `Kaggle` 上提供为学习和实践数据分析的资源。该数据集包含了一个小型商业公司的各种业务数据，如产品、订单、客户、员工等信息。它是一个关系型数据库的经典示例，包含了多个相互关联的表格。以下是 `Northwind Traders` 数据库中通常包含的一些主要表格及其字段说明：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Customers (客户表)</font></p>
<div class="center">

| 字段名称     | 数据类型     | 描述           |
|--------------|--------------|----------------|
| CustomerID   | nchar(5)     | 客户ID (主键)  |
| CompanyName  | nvarchar(40) | 公司名称       |
| ContactName  | nvarchar(30) | 联系人姓名     |
| ContactTitle | nvarchar(30) | 联系人职务     |
| Address      | nvarchar(60) | 地址           |
| City         | nvarchar(15) | 城市           |

</div>

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Employees (员工表)</font></p>
<div class="center">

| 字段名称        | 数据类型     | 描述           |
|-----------------|--------------|----------------|
| EmployeeID      | int          | 员工ID (主键)  |
| employeeName        | nvarchar(20) | 员工名字           |
| Title           | nvarchar(30) | 职位           |
| City            | nvarchar(15) | 城市           |
| Country         | nvarchar(15) | 国家           |
| ReportsTo       | int          | 上级ID (外键)  |

</div>

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4> Products (产品表)</font></p>
<div class="center">

| 字段名称        | 数据类型     | 描述               |
|-----------------|--------------|-------------------|
| ProductID       | int          | 产品ID (主键)      |
| ProductName     | nvarchar(40) | 产品名称           |
| CategoryID      | int          | 类别ID (外键)      |
| QuantityPerUnit | nvarchar(20) | 单位数量           |
| UnitPrice       | money        | 单价               |
| Discontinued    | bit          | 是否停产           |

</div>

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Categories (分类表)</font></p>
<div class="center">

| 字段名称     | 数据类型     | 描述           |
|--------------|--------------|----------------|
| CategoryID   | int          | 类别ID (主键)  |
| CategoryName | nvarchar(15) | 类别名称       |
| Description  | ntext        | 类别描述       |

</div>

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Shippers (货运公司表)</font></p>
<div class="center">

| 字段名称    | 数据类型     | 描述                 |
|-------------|--------------|----------------------|
| ShipperID   | int          | 货运公司ID (主键)    |
| CompanyName | nvarchar(40) | 公司名称             |

</div>

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Orders (订单表)</font></p>
<div class="center">

| 字段名称       | 数据类型     | 描述                 |
|----------------|--------------|----------------------|
| OrderID        | int          | 订单ID (主键)        |
| CustomerID     | nchar(5)     | 客户ID (外键)        |
| EmployeeID     | int          | 员工ID (外键)        |
| OrderDate      | datetime     | 订单日期             |
| RequiredDate   | datetime     | 要求交货日期         |
| ShippedDate    | datetime     | 实际发货日期         |
| ShippedID        | int          | 运输方式ID (外键)    |
| Freight        | money        | 运费                 |

</div>

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Order Details (订单详情表)</font></p>
<div class="center">

| 字段名称  | 数据类型 | 描述                      |
|-----------|----------|---------------------------|
| OrderID   | int      | 订单ID (主键部分/外键)    |
| ProductID | int      | 产品ID (主键部分/外键)    |
| UnitPrice | money    | 单价                      |
| Quantity  | smallint | 数量                      |
| Discount  | real     | 折扣                      |

</div>

&emsp;&emsp;以上是`Northwind Traders` 数据库中主要表格的关联关系，大家可以直接在其地址为：https://www.kaggle.com/datasets/jeetahirwar/northwind-traders 下载到本地。不过，我们这里仿照该数据集，通过脚本生成了类似结构的符合电商领域客服场景的数据集，构建脚本存储位置在：`graphrag\origin_data\create_sql_data.py`，大家可以直接运行该脚本，生成符合我们需求的数据集。


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211529006.png" width=60%></div>

&emsp;&emsp;这份脚本中是以`Northwind`结构示例构建了一个完整的电子商务业务场景，包括产品目录、客户信息、订单系统、员工信息、产品评论等一系列表结构，使用`Faker` 库生成符合电商领域客服场景的数据，并使用`SQLAlchemy` 库将数据存储到`MySQL` 数据库中。

&emsp;&emsp;因此，运行脚本前，需要<font color=red>**保证先启动`Mysql` 服务，并创建好对应的数据库，然后，在`create_sql_data.py` 脚本中，修改对应的数据库连接信息后，再执行运行**</font>。除此以外，在`create_sql_data.py` 的最后，还可以通过灵活的配置来生成不同数量级的数据，如下所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211537961.png" width=60%></div>

&emsp;&emsp;数据库配置信息及数据量参数配置好以后，即可以通过如下命令运行脚本：

```bash
    cd .\origin_data\
    python .\create_sql_data.py
```

&emsp;&emsp;运行脚本后，在控制台终端可以看到具体的输出信息，如下所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211539551.png" width=80%></div>

&emsp;&emsp;同时，在`MySQL` 数据库中，可以看到生成的一系列数据表，如下所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211542929.png" width=60%></div>

&emsp;&emsp;除了在`MySQL` 数据库中可以看到生成的一系列数据表，在`graphrag\origin_data\exported_data` 目录下，还可以看到生成的一系列数据表的`.csv` 文件，如下所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211544209.png" width=60%></div>

&emsp;&emsp;至此，我们就准备好了完整的智能客服数据集，接下来，我们就可以开始使用`Microsoft GraphRAG` 来构建智能客服的知识图谱了。

# 2. 使用Microsof GraphRAG 处理非结构化.csv数据

&emsp;&emsp;`.csv` 文件格式存储的数据本质上就是结构化数据，例如`Mysql`、`Mongodb` 数据库中存储的数据可以直接导出为`.csv` 文件。知识图谱概念最开始出现的时候，就是以结构化数据为基础，`.csv` （或数据库）中存储的数据，天然就适合用于知识图谱的构建。知识图谱最核心的是节点和边，而`.csv` 文件中各列的含义可以映射到知识图谱中的节点和边。例如，某一列可以表示实体（节点），而另一列可以表示实体之间的关系（边）。通过外键和约束，可以在关系型数据库中定义这些关系，这与知识图谱的构建逻辑相似。

&emsp;&emsp;这里大家要清晰一个概念：`Microsof GraphRAG` 构建索引的过程是在做什么？

&emsp;&emsp;本质上，`Microsof GraphRAG` 构建索引就是要抽取出这些节点和边去构建知识图谱，那么`.csv` 文件中存储的结构化数据本身就能够很好的体现出节点和边的关系，是没有必要再去重复做`GraphRAG`的`Index`过程的，且效果大概率也不会比直接使用`.csv` 文件中的数据构建知识图谱的效果更好。因此，如果业务数据表特别特别多（几十到几百张表），对应的混合检索策略就是：

1. 对于非自然语言的结构化数据，直接定义明确的规范构建知识图谱，导入到如`Neo4j` 中进行统一管理。比如以客服场景为例，能从结构化数据中梳理出来的明确节点和边如下所示：

```json
    - 节点类型
        - Product - 产品节点
        - Supplier - 供应商节点
        - Customer - 客户节点
        - Employee - 员工节点
        - Shipper - 物流公司节点
        - Order - 订单节点
    - 边类型
        - SUPPLIED_BY - 产品由供应商提供
        - PLACED_BY - 订单由客户下单
        - PROCESSED_BY - 订单由员工处理
        - SHIPPED_VIA - 订单通过物流公司配送
        - CONTAINS - 订单包含产品
        - REPORTS_TO - 员工上下级关系
```

&emsp;&emsp; 2. 对于自然语言的非结构化数据，根据场景的特定信息做文本聚合。比如如果想要提取一些商品的评论信息，可以按照商品的名称进行聚合，然后对聚合后的内容进行实体和关系的提取。比如：

| ReviewID | ProductID | ProductName | CustomerID | CustomerName | Rating | Text                                                         | ReviewDate | CategoryName |
|----------|-----------|-------------|------------|--------------|--------|--------------------------------------------------------------|------------|--------------|
| 22       | 5         | LG 智能门铃  | NI721      | 泰麒麟科技有限公司数字科技有限公司 | 4      | 智能门铃的设计很人性化，夜视效果好，值得推荐。 | 2025-03-16 | 智能门铃     |

&emsp;&emsp;聚合后的格式为：

```json
    "客户 {CustomerName} 对 {CategoryName} 产品 {ProductName} 的评价 (评分: {Rating} 星): \"{Text}\" (评价日期: {ReviewDate})"
```

&emsp;&emsp; 后者就是典型的`Microsof GraphRAG`比较适合的解析领域，而在前面的课程中，我们已经介绍过如何把`Microsof GraphRAG`生成的一系列`parquet` 文件导入到`Neo4j` 中，所以混合检索的策略也非常多，主要有如下几种：

1. 将直接构建的`Neo4j` 知识图谱与`Microsof GraphRAG` 构建出来的图谱进行合并，用统一的检索流程进行问答检索；
2. 分离直接可以构建的`Neo4j` 知识图谱和`Microsof GraphRAG` 构建出来的图谱，用不同的检索流程进行问答检索；
   - 直接可以构建的`Neo4j` 知识图谱，使用 `TEXT2CYPHER` 的流程进行问答检索；
   - `Microsof GraphRAG` 构建出来的图谱，使用 `Global Search` 或者 `Local Search` 的流程进行问答检索；

&emsp;&emsp; 实测效果比较好的是第二种策略，其一是可以按照业务场景分离数据库，各自建立对应的检索优化流程；其二在知识库的后期维护中，可以按照业务场景分离数据库，隔离数据。同时，借助`Agent`技术做好第一轮的意图识别，根据用户需求选择对应的数据库进行针对性的检索，也可以有效降低检索的响应时长。

&emsp;&emsp; 接下来，我们就来实际的介绍一下在`Microsof GraphRAG` 中处理`.csv` 文件的处理和优化策略。但需要说明的是，在进行策略优化之前，我们先需要了解一下`Microsof GraphRAG` 默认的处理`.csv` 文件的工作原理。

## 2.1 默认的加载与处理.csv文件策略

&emsp;&emsp; 注意：课程当前使用的`Microsof GrapRAG` 版本是 `v2.1.0` 。

&emsp;&emsp; 在目前最新的`Microsof GrapRAG v2.1.0` 版本中，默认支持 `.csv` 文件的批量处理，并且其构建索引的逻辑与我们在前几节课介绍的`.txt` 文件的构建索引的逻辑是一样的。但需要重点了解的是：每个 `.txt` 文件中的全部内容，会被看做是一个完整的 `Document` 进行导入，同时每个 `Document` 会具有一个唯一的 `id` 。即每个`.txt` 文件对应一个 `document id`，以`document id` 为单位进行后续的 `text_unit` 切分、实体提取、社区报告生成等一系列 `Workflow` 。

&emsp;&emsp; 但对于 `.csv` 文件来说，`.csv` 文件的特性是：每个`.csv` 文件中会有很多`Row`（行）, 每个`Row` 中会包含很多`Column` （列），每个 `Column` 表示的含义也不一样。那很明显是<font color="red">没有办法像`.txt` 文件那样，把一个 `.csv` 文件中的全部内容，作为唯一的 `Document` 进行后续处理的。</font> 因此，我们需要重点了解 `Microsof GraphRAG` 中的 `.csv` 文件的处理逻辑才能针对性的根据`.csv` 文件中的内容进行精确地知识图谱的构建。

&emsp;&emsp; 首先来看下，在`Microsof GraphRAG` 中如果要处理`.csv` 文件，可以在`settings.yaml` 中配置哪些参数。具体如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>CSV 文件处理相关字段</font></p>
<div class="center">

| 字段名称       | 描述                                   | 默认值                                      |
|----------------|----------------------------------------|---------------------------------------------|
| `file_type`    | 指定要使用的输入文件类型。            | `file` |
| `base_dir`     | 指定输入文件的基本目录。              | `input`  |
| `file_pattern` | 指定输入文件的模式（例如，文件名模式）。| `.*\\.csv$$` |
| `file_filter`  | 可选的文件过滤器，用于输入文件。      | `None` |
| `text_column`  | 指定用于输入文本的列。                | `None` |
| `title_column` | 指定用于输入标题的列（可选）。        | `None` |
| `metadata`     | 指定用于文档属性的列（可选）。        | `None`   |

</div>

&emsp;&emsp; 以上参数可以灵活的配置`.csv`文件的读取逻辑，接下来我们需要了解 `Microsof GraphRAG` 中，对于 `.csv` 文件的处理逻辑，其过程如下：

1. 首先，通过 `storage` 根据 `settings.yaml` 中设置的文件存放路径、切分策略等找到需要进行处理的 `csv` 文件。
2. 通过 `pd.read_csv` 读取 `csv` 文件，更具体的处理策略是：
    - 使用哈希函数，<font color="red">对 `csv` 文件中的每一行生成唯一标识符，作为 `document id` 。</font>
    - 选择`.csv`文件中的某一列，作为该`document id` 的文本内容，用于后续的`text_unit` 切分、实体提取、社区报告生成等一系列 `Workflow` 。选择的策略是：
      - 如果传入的`.csv`文件中，存在 `text` 列，则选择`text` 列作为文本内容。
      - 如果传入的`.csv`文件中，不存在 `text` 列，则选择在`settings.yaml` 中配置的 `text_column` 列作为文本内容。
      - 如果既没有 `text` 列，也没有配置 `text_column` 列，则记录警告信息，且终止Workflow。

&emsp;&emsp; 如下图所示：（源码位置：graphrag\graphrag\index\input\csv.py）

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503201422667.png" width=80%></div>

&emsp;&emsp; 上述规则是保证`Microsof GraphRAG` 能够正常加载`.csv` 文件且能够正常进行后续的`Workflow` 的前提条件。

&emsp;&emsp; 除此以外，对于批量处理`.csv` 文件来说，其潜在的规则是：<font color="red">当在`input` 目录下有多个`.csv` 文件时，如果首个`.csv` 文件中有`text` 列，或者在`settings.yaml` 中配置了 `text_column` 列仅包含在首个`.csv` 文件中，那么一旦后面的`.csv` 文件中不存在`text` 列，或者不存在`settings.yaml` 中配置的 `text_column` 列，那并不会报错，而是全部加载为`None`，即会丢失信息。如下所示：</font>


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503201448084.png" width=80%></div>

&emsp;&emsp; 因此，在实际使用中，需要保证的是：<font color="red">在`input` 目录下所有`.csv` 文件中，都需要有统一配置的`text` 列，或者`text_column` 列，才可以正常的加载到文本内容，否则会丢失信息。</font>

&emsp;&emsp; 在了解了以上两个关键点以后，能够保证的是：可以把所以`.csv` 文件中的内容正常加载并且执行索引构建的流程。比如我把每个`.csv` 文件中需要进行解析的`Column` 字段名定义为`text`，完整的索引构建流程即可正常的运行。如下所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503201527417.png" width=80%></div>

&emsp;&emsp; 这个阶段关注以下两点，会直接影响到后续的索引构建流程：

- **1. 切分策略**

&emsp;&emsp; `.csv` 文件是每一行作为一个`Document`，所以其`text_unit` 切分策略，其实就是对`text` 列执行在`settings.yaml` 中配置的 `size` 和 `overlap` 参数，默认是按照`Token` 切分。所以这里存在的问题是：

1. 如果你的 `text` 列中的内容特别短，单独作为一个`Document` 进行索引构建，就会导致索引构建的流程特别慢，效率低且会大量消耗额外的费用或算力。
2. 一个`csv` 表格中，仅有`text`这一列的内容会被用于构建索引提供有效信息，无法识别到其他的`Column` 列中的内容。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503201555733.png" width=80%></div>

&emsp;&emsp;同样，实体、关系提取、社区发现等`Workflow` 都是以 `text_unit` 为单位进行处理的，所以如果`text_unit` 如果不能包含丰富的信息和有效的切分策略，后续的流程中自然会进一步丢失更多有价值的信息，构建出来的知识图谱一定无法提供有效的检索。


&emsp;&emsp;这是非常明显的问题，但不了解`MicroSoft GraphRAG`底层原理的话，一定想不到在第一步`Document` 切分的时候，就已经丢失了大量的信息。所以在实际的使用中，针对`csv` 文件类型，一种有效的策略是做混合检索。


## 2.2 csv 文件加载优化策略

&emsp;&emsp; 熟练掌握了上述的`Microsof GraphRAG` 默认的加载与处理`.csv` 文件的策略后，接下来，我们就可以开始尝试做针对性的优化策略了。

&emsp;&emsp; 首先，我们需要根据自己的业务需求，在偌大的业务数据中构建出较为完整的语义文本。以智能客服场景为例，我们可以抽象出如下几种业务需求通常是以自然语言存储的：

1. 客户评论与反馈数据：
```csv
    review_id,product_id,customer_id,rating,review_text,date
    101,P234,C567,4,"这款智能音箱音质非常好，但连接蓝牙设备时偶尔会断开。语音助手理解能力还需提高。",2023-05-12
```
&emsp;&emsp; 其业务价值是从评论文本中提取产品特性、问题点和情感, 构建产品-特性-问题的知识网络, 帮助客服快速了解产品常见问题。


2. 客服聊天记录
```csv
    conversation_id,timestamp,sender_type,message,intent,order_id
    CS1001,2023-06-15 14:23:45,customer,"我上周五买的小米智能灯泡无法连接到我的华为手机，我已经按照说明书重置了三次",product_issue,ORD8976
```

&emsp;&emsp;其业务价值是提取产品问题、解决方案和兼容性信息，构建问题-故障-解决方案的知识图谱，训练客服系统自动回答类似问题。


3. 退货和投诉记录
```csv
    case_id,order_id,product_id,return_reason,customer_description,resolution
    RET345,ORD7891,P456,"产品损坏","收到的智能插座有明显的外壳裂痕，插入电源后指示灯闪烁红色，无法正常工作","退款并更换新品"
```
&emsp;&emsp;其业务价值是提取产品问题模式和解决方案， 构建产品-问题类型-解决方案关系。帮助客服系统自动处理退货和投诉。

&emsp;&emsp;有了业务需求后，开始做数据处理，这个过程没什么捷径，就是根据业务需求，去数据表中找到对应的字段，编写代码脚本匹配成一个个完整的自然语言表示。因为实体提取，我们把一些 `ProductID`、`CustomerID`、`CategoryID` 等字段给到大模型，大模型是无法识别出的。但是当转化成自然语言后，就可以把具体的产品、客户、类别等实体给提取出来，而精准的实体名，是可以在知识图谱中根据某些`ID`的属性找到的。这里我以评论数据场景为例，其处理过程如下所示：

&emsp;&emsp;原始数据：

| ReviewID | ProductID | ProductName     | CustomerID | CustomerName                     | Rating | Text                                               | ReviewDate | CategoryName |
|----------|-----------|------------------|------------|----------------------------------|--------|----------------------------------------------------|------------|--------------|
| 10       | 5         | 索尼 智能体重秤 Basic | MH412      | 创联世纪传媒有限公司智能科技      | 4.9    | 这款智能体重秤质量很好，反应灵敏。总体来说很满意。 | 2024-06-12 | 智能体重秤   |

``` json
    处理后的 text 字段中的内容：

    客户ID: MH412
    客户公司: 创联世纪传媒有限公司智能科技
    客户所在地: 文市, 中国
    产品信息: 索尼 智能体重秤 Basic (类别: 智能体重秤)
    生产商: 索尼智能科技
    产品价格: 2042.82
    评分: 4.9星
    评价日期: 2024年06月12日
    评价内容: "这款智能体重秤质量很好，反应灵敏。总体来说很满意。
```

&emsp;&emsp; 构建完整的`text`语义信息是第一步，接下来结合`Microsof GraphRAG` 的`Workflow` 流程，我们还需要进行策略上的优化，具体来说：就是把单条语义信息合并成更大的文本块，从而能在构建索引的单次处理中，尽可能的包含更多的信息，降低构建成本的同时，大幅提升索引构建的效率。在执行这一步的时候，需要考虑的是尽可能的把语义相近的文本块合并在一起，因此相关的策略可参考如下：

1. **分组**：通过`groupby` 分组，例如，所有属于“电子产品”的评论、产品或数据可以被分在一起。这种分组有助于在分析时保持上下文的一致性。
2. **设置最大Token**：在每个类别内，根据`token`数量动态决定每批次包含的评论数量，避免某些未知的评论过长，超出最大上下文限制，同时太长的文本框会在读取时导致`OOM` 错误。
3. **设置分隔符**：在合并多行时，通过特殊字符如`<ROW_SEP>`来标记单独行之间的边界，用于在`Microsof GraphRAG` 的`text_unit` 切分中，能够正确识别出单独的行，制定单独的切分策略。


&emsp;&emsp;完整的脚本文件存储位置为`graphrag\origin_data\preprocess_data.py`, 最终生成的文件为`merged_data.csv`。 如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503201916252.png" width=80%></div>

&emsp;&emsp; 最终用于构建索引的文件为`merged_data.csv`，其单列内容如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503201929240.png" width=80%></div>

&emsp;&emsp; 大家可以按照相同的思路构建多个`.csv`文件来支撑不同的业务需求，唯一需要保证的是：把构建出来最终的文本框的`Column` 名全部设置为`text`，然后放到`input` 文件夹中，`Microsof GraphRAG` 会可以自动识别到并且顺利开始索引构建流程。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503201933232.png" width=80%></div>

&emsp;&emsp;在`settings.yaml` 文件中的`input` 中设置加载`.csv`文件，指定`text_column` 为`text`，然后，在`chunks` 中设置文本的切分策略，主要有`size` 和`overlap` 两个参数，`size` 表示每个文本块的最大长度，`overlap` 表示相邻文本块之间的重叠长度，默认`.csv`文件是以`token`进行切分的。全部配置好以后，执行如下命令开始构建索引：

```bash
    poetry run poe index --root ./data
```

&emsp;&emsp;注意，这里的`./data` 是因为我在初始化时候使用`poetry run poe init --root ./data` 初始化的项目，所以索引构建的根目录为`./data`，大家根据自己的实际情况设置即可。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211005809.png" width=80%></div>

&emsp;&emsp;这里的切分策略是：对`.csv` 文件中的每一行`row`(行)， 会看做一个`Document`，按照设置的`size` 和`overlap` 进行切分，所以这种切分方法是存在一定问题的，主要在于：为了增加索引效率，我们手动将多行合并成一个`text`，如果按照这种切分方法，极大概率会把某一行原始的`text`切分开，导致这一行的信息丢失，从而影响索引构建的效果，基于此，我们需要手动实现对`.csv` 文件的切分策略优化。

## 2.3 csv 文件切分优化策略

&emsp;&emsp;结合`Microsof GraphRAG` 的内置流程和我们在前一阶段对`.csv` 文件的预处理，一种非常行之有效的切分优化策略如下：

1. 首先使用<ROW_SEP>分隔符拆分文本，每个分隔后的部分作为一个完整的评论块处理；
2. 对于正常长度的评论块，按照 `config.size` 进行分组。假设一个text中有两个评论块，每个token数为20，而config.size设置为30，切分策略如下：
   - 首先检查第一个评论块：
        - 当前chunk为空，所以直接添加第一个评论块（20 tokens）
        -  current_chunk_texts = [评论块1]
        - current_chunk_size = 20
   - 然后检查第二个评论块：
        - 检查添加后是否会超过限制：current_chunk_size + row_tokens = 20 + 20 = 40
        - 因为40 > 30（config.size），所以会先保存当前chunk
        - 创建一个包含第一个评论块的chunk并添加到结果中
        - 然后重置当  前chunk，并添加第二个评论块
        - current_chunk_texts = [评论块2]
        - current_chunk_size = 20
   - 最后处理：
        - 处理完所有评论块后，保存最后一个chunk（包含第二个评论块）
        - 所以最终结果是两个chunk，每个包含一个评论块。

2. 当单个评论块超过`config.size`时，借助`chunk_overlap` 参数尽可能的保留完整的语义信息。

&emsp;&emsp; 上述优化策略的实现代码已经集成到`Microsof GraphRAG` 源码中，存储位置为：graphrag\graphrag\index\operations\chunk_text\csv_strategy.py。 同时，如果要启用我们自定义的切分策略，仅需要修改`settings.yaml` 文件中`chunks` 下的 `strategy` 参数为`csv` 即可，如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211106448.png" width=80%></div>

&emsp;&emsp; 通过输出日志也可以看到，优化后的切分策略则不会严格按照`size` 和`overlap` 进行切分，而是可以更加灵活的根据语义信息进行切分，从而保留单个块的完整语义信息。

&emsp;&emsp; 至此，我们现在已经优化了`.csv` 格式文件的预处理和`text_unit` 的切分，接下来的`Workflow` 流程将是对每一个`text_unit` 进行实体和关系的提取。因此，我们要进一步优化的就是`Prompt` 模板，从而让大模型可以更加精准的提取出我们需要的实体和关系。

## 2.4 实体关系提取优化策略

&emsp;&emsp; 在`Microsof GraphRAG` 中，实体、关系、社区报告、摘要等信息的提取都是依赖`Prompt` 模板 + 大模型来生成的。其中，用于实体关系提取的`Prompt` 模版为`extract_graph.txt`，其内容如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211140616.png" width=80%></div>

&emsp;&emsp; 同时，仅使用`extract_graph.txt` 模板其默认提取的实体和关系类型为：`organization`、`person`、`geo`、`event`。这个很明显是不太符合我们传入的智能客服场景的，因此，我们需要设计一个更加符合我们业务需求的`Prompt` 模板。

&emsp;&emsp; `Microsof GraphRAG`  提供了两种方式来调整`Prompt` 模板，分别是手动调整和自动调整。其中：手动调整指的是通过以纯文本形式编写自定义提示文件来覆盖每个提示，但有一定的限制，即需要包含指定的关键词，稍有不慎就会导致`Workflow` 流程中断。另外一种方式是自动调整，即通过`Microsof GraphRAG` 内置实现的 `Prompt` 模板优化器来调整`Prompt` 模板，这种方式也是我们推荐的方式。

&emsp;&emsp; 接下来我们就依次介绍两种优化方式。首先来看 `Auto Prompt Tuning` 优化方式。

&emsp;&emsp;首先，`Auto Prompt Tuning` 的工作流入口函数仍然是在`graphrag\graphrag\cli\main.py`文件中的`_prompt_tune_cli`函数，位置如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211307155.png" width=80%></div>

&emsp;&emsp; 其可用的参数如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Auto Prompt Tuning 优化参数</font></p>
<div class="center">

| 参数名                     | 描述                                                                                     | 状态   | 默认值       |
|--------------------------|----------------------------------------------------------------------------------------|------|------------|
| --config                 | 配置文件的路径。这是加载数据和模型设置所必需的。                                            | 必需   | 无         |
| --root                   | 数据项目根目录，包括配置文件（YML、JSON 或 .env）。                                          | 可选   | 当前目录     |
| --domain                 | 与输入数据相关的域，例如“空间科学”、“微生物学”或“环境新闻”。如果留空，则将从输入数据中推断出域。 | 可选   | 无         |
| --selection-method       | 选择文档的方法。选项包括 all、random、auto 或 top。                                      | 可选   | random     |
| --limit                  | 使用随机或顶部选择时要加载的文本单元的限制。                                             | 可选   | 15         |
| --language               | 用于输入处理的语言。如果它与输入的语言不同，则将进行翻译。默认值为“”，表示将自动从输入中检测。 | 可选   | 自动检测 |
| --max-tokens             | 提示生成的最大令牌数。                                                                    | 可选   | 2000       |
| --chunk-size             | 用于从输入文档生成文本单元的标记大小。                                                  | 可选   | 200        |
| --n-subset-max           | 使用自动选择方法时嵌入的文本块数量。                                                    | 可选   | 300        |
| --k                      | 使用自动选择方法时要选择的文档数量。                                                    | 可选   | 15         |
| --min-examples-required   | 实体提取提示所需的最小示例数。                                                            | 可选   | 2          |
| --discover-entity-types   | 允许自动发现和提取实体。当数据涵盖大量主题或高度随机化时，建议使用此选项。           | 可选   | 无         |
| --output                 | 保存生成的提示的文件夹。                                                                  | 可选   | prompts |

</div>

&emsp;&emsp;接下来我们依然借助源码来详细介绍`Auto Prompt Tuning` 的实现流程，以及上述各个参数的实际应用价值。其中`prompt tune` 的源码核心函数位于：`graphrag\graphrag\api\prompt_tune.py` 文件中的`generate_indexing_prompts`函数，如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211311625.png" width=80%></div>

&emsp;&emsp; 其核心流程包含以下 11 个步骤：

1. 解析`settings.yaml` 中配置的参数， 通过 `--config` 参数；
2. 找到在 `input` 文件夹中存放的文档， 通过 `--root` 参数指定根目录；
3. 将读取到的文档切分成`text_unit` （块），通过 `--chunk_size` 参数指定切分策略中每个块的大小；
4. 限制用于后续生成提示的原始文档的数量，通过 `--limit` 参数指定；
5. 根据 `--k` 参数，进一步选择具体的`text_unit` 块，包含三个策略：
    - 如果 `--selection-method` 参数为 `top`，则选择前`k`个文本单元；
    - 如果 `--selection-method` 参数为 `random`，则随机选择`k`个文本单元；
    - 如果 `--selection-method` 参数为 `auto`，则使用`k`和`n_subset_max`选择文本单元，具体的实现逻辑是：
        - `--n-subset-max` 用来限制随机采样的取值，表示从`text_unit`个文本单元中随机采样`n_subset_max`个文本单元，转化成 `Embedding` 向量；
        - 计算采样文本单元的 `Embedding` 向量平均值，即中心点；
        - 计算每个采样文本块到中心点的距离，选择出`k`个距离中心点最近的文本单元；
        - 最终选择出来的文本最能代表整个文档集合的一般特征，同时可以过滤掉一些极端的文本块；
6. 设置输入数据所属的场景领域，比如`电商客服`、`算法分析`等，用于填充`Prompt` 模板中的`domain` 参数，如果未指定，则通过内置提示词 + 语言模型自动生成，最后填充 `Prompt` 模板中的`domain` 参数；
7. 设置最终输出的提示词模版的语言，通过`--language` 参数指定，如果未指定，则通过内置提示词 + 语言模型自动生成，最后填充 `Prompt` 模板中的`language` 参数；
8. 根据设置的领域`domain`，生成模型的系统身份背景信息；
9. 如果设置 `--discover_entity-types`， 则会通过提示词 + 语言模型自动发现符合当前输入数据的实体类型；
10. 生成用于生成实体配置的实体/关系示例列表，用来填充`Prompt` 模板中的`examples` 参数，即`Few-shot` 示例;
11. 最后，依次生成用于提取实体、关系、社区报告、摘要的提示词，可以通过 `--min_examples-required` 参数指定每个提示词需要的最小示例数量，以及通过`--output` 参数指定输出文件夹。

&emsp;&emsp; 通过以上的源码解读，我们则可以非常清楚的了解如何执行`Auto Prompt Tuning` 流程。因此在源码环境下，我们可以使用如下命令来执行`Auto Prompt Tuning` 流程：

```bash
    poetry run poe prompt_tune --config ./data/settings.yaml --root ./data --domain "智能客服"  --selection-method top --k 10 --output ./data/prompt_turn_output
```
&emsp;&emsp; 如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211402436.png" width=80%></div>

&emsp;&emsp; 最终生成微调后的`Prompt` 模板文件，将存储在我们通过`--output` 参数指定的文件夹中，如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211602284.png" width=80%></div>

&emsp;&emsp; 这里生成的`Prompt` 模板文件就是针对我们特定智能客服场景下以及具体的数据，借助大模型生成的符合 `Microsoft GraphRAG` 索引构建`Workflow` 的提示词。如果想要使用最新的`Prompt` 模板文件，则需要将在`settings.yaml` 文件中配置的`prompt_template_path` 参数指向最新的`Prompt` 模板文件。即如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211633770.png" width=80%></div>

&emsp;&emsp; 修改好配置后，我们就可以使用最新的`Prompt` 模板文件来构建`Microsoft GraphRAG` 索引了。仍然是使用`poetry run poe index` 命令来构建`Microsoft GraphRAG` 索引，如下所示：

```bash
    poetry run poe index --root ./data/
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211634453.png" width=80%></div>

&emsp;&emsp; 全部执行结束后，依然会正常在`./data/output` 文件夹中生成对应的`.parquet` 文件，可以直接用于`Microsoft GraphRAG` 的检索过程，同时，如果大家想存储到`Neo4j` 数据库中进行可视化展示，也可以按照我们在`《Part 3. Microsoft GraphRAG 自定义接入图数据库 Neo4j》` 课程中的介绍进行持久化的存储。

## 3. 结构化数据直接导入`Neo4j` 的方法

&emsp;&emsp;上述的流程我们操作的是非结构化数据，构建的方式是通过`提示工程+大模型` 借助`Microsoft GraphRAG` 的`Workflow` 来构建的。如果大家想操作结构化数据，则需要结合实际的业务场景，将数据集拆分为节点和边，并按照节点类型和边类型分类，符合Neo4j等图数据库的标准导入方式。

&emsp;&emsp;这个过程其实没什么技巧可谈，一切以业务需求为驱动，考验的就是大家的业务理解能力和数据处理能力，以及知识图谱的构建能力。 比如，我们以`电商智能客服` 为例，可以先分析一下智能客服场景的需求，主要的点包括：

1. 需要快速检索产品信息
2. 需要了解客户的基本信息和订单历史
3. 需要了解产品与供应商、类别的关系
4. 需要追踪订单状态和物流信息


&emsp;&emsp;基于以上需求，我们就可以开始着手划分`Neo4j` 的节点和边。其中：

- 节点类型
    - Product - 产品节点
    - Category - 产品类别节点
    - Supplier - 供应商节点
    - Customer - 客户节点
    - Employee - 员工节点
    - Shipper - 物流公司节点
    - Order - 订单节点
    
- 边类型
    - BELONGS_TO - 产品属于类别
    - SUPPLIED_BY - 产品由供应商提供
    - PLACED_BY - 订单由客户下单
    - PROCESSED_BY - 订单由员工处理
    - SHIPPED_VIA - 订单通过物流公司配送
    - CONTAINS - 订单包含产品
    - REPORTS_TO - 员工上下级关系

&emsp;&emsp; 构建导入的方案细节，如下图所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>节点类型</font></p>
<div class="center">

| 节点类型   | 标识          | 属性                                                                                     |
|------------|---------------|------------------------------------------------------------------------------------------|
| Product    | ProductID     | ProductName, QuantityPerUnit, UnitPrice, UnitsInStock, UnitsOnOrder, ReorderLevel, Discontinued |
| Category   | CategoryID    | CategoryName, Description                                                                 |
| Supplier   | SupplierID    | CompanyName, ContactName, ContactTitle, Address, City, Region, PostalCode, Country, Phone, Fax, HomePage |
| Customer   | CustomerID    | CompanyName, ContactName, ContactTitle, Address, City, Region, PostalCode, Country, Phone, Fax |
| Employee   | EmployeeID    | LastName, FirstName, Title, BirthDate, HireDate, Address, City, Country, HomePhone, Notes |
| Shipper    | ShipperID     | CompanyName, Phone                                                                        |
| Order      | OrderID       | OrderDate, RequiredDate, ShippedDate, Freight, ShipName, ShipAddress, ShipCity, ShipRegion, ShipPostalCode, ShipCountry |

</div>

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>关系类型</font></p>
<div class="center">

| 边类型         | 源节点      | 目标节点    | 数据源                                                                                  |
|----------------|-------------|-------------|-----------------------------------------------------------------------------------------|
| BELONGS_TO     | Product     | Category    | products.csv (ProductID -> CategoryID)                                                 |
| SUPPLIED_BY    | Product     | Supplier    | products.csv (ProductID -> SupplierID)                                                |
| PLACED_BY      | Customer    | Order       | orders.csv (CustomerID -> OrderID)                                                    |
| PROCESSED_BY   | Employee    | Order       | orders.csv (EmployeeID -> OrderID)                                                   |
| SHIPPED_VIA    | Order       | Shipper     | orders.csv (OrderID -> ShipVia)                                                       |
| CONTAINS       | Order       | Product     | order_details.csv (OrderID -> ProductID)                                             |
| REPORTS_TO     | Employee    | Employee     | employees.csv (EmployeeID -> ReportsTo)                                               |
</div>

&emsp;&emsp;定义好构建的规范后，便可以开始实际的构建知识图谱。对于单独的`.csv`文件，或者从数据库直接导出的`.csv`文件，一种最高效处理大批量数据的方法是使用`Neo4j` 的`Neo4j Admin` 工具。`Neo4j Admin` 是 `Neo4j` 数据库管理系统中的一个命令行工具，主要用于管理和维护 `Neo4j` 数据库实例。它提供了一系列命令，帮助执行各种管理任务，如数据库的创建、备份、恢复、配置和监控等。

&emsp;&emsp;如果采用的是我们在`《Part 3. Microsoft GraphRAG 自定义接入图数据库 Neo4j》` 课程中的介绍的源码安装方式，则`Neo4j Admin` 工具位于`neo4j-admin` 文件夹中。


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211654165.png" width=80%></div>

&emsp;&emsp;但是需要明确的是，`Neo4j Admin` 可以直接导入`.csv` 文件，但如何去处理原始文件中的节点和关系，则需要我们通过注释这一概念来进行声明。

&emsp;&emsp;`Neo4j` 中，注释（或称为注释行）是用于在 `CSV`文件中提供元数据的行。这些注释行以 # 开头，允许开发者为每个字段指定类型、约束和其他信息，以便在导入数据时，`Neo4j` 能够正确解析和处理这些数据。其中 `CSV` 注释的用途包括：

1. 字段类型: 指定字段的数据类型（如字符串、整数、布尔值等），确保 `Neo4j` 在导入时能够正确识别字段类型。
2. 约束: 指定唯一性约束、索引等，以确保数据的完整性和一致性。
3. 关系类型: 指定边的类型，确保在导入时正确建立节点之间的关系。

&emsp;&emsp;对应的注释的格式是需要在`CSV` 文件的开头添加注释行，格式如下：

```json
    # Node: Product
    # Field: ProductID, Type: Integer, Unique: true
    # Field: ProductName, Type: String
    # Field: UnitPrice, Type: Float
    # Field: UnitsInStock, Type: Integer
    # Field: CategoryID, Type: Integer, Relationship: BELONGS_TO

    
    1,智能音箱,299.99,100,1
    2,智能灯具,199.99,150,2
    3,智能插座,89.99,200,1
```

&emsp;&emsp;这里我已经把`Neo4j`的结构定义以及对应的注释行都写好了，全部流程封装在`graphrag\origin_data\create_neo4j_import.py` 文件中，大家可以直接运行该文件来生成`Neo4j` 的导入文件。


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211704385.png" width=80%></div>

&emsp;&emsp; 运行方法如下图所示:

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211710866.png" width=80%></div>

&emsp;&emsp; 运行结束后，会在`graphrag\origin_data\neo4j_admin` 文件夹中生成对应的`Neo4j` 的导入文件即导入脚本。然后我们需要把`import_command.bat` 文件放在`Neo4j` 安装目录下的`bin` 文件夹中,如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211719459.png" width=80%></div>

&emsp;&emsp; 然后打开`cmd` 命令行，进入到`Neo4j` 安装目录下的`bin` 文件夹中，运行`import_command.bat` 文件，即可开始导入数据。导入过程如下图所示：注意：执行导入之前需要先停止`Neo4j` 服务，否则会报错。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211719460.png" width=80%></div>

&emsp;&emsp; 等待执行完成后，再次启动`Neo4j` 服务，就可以在`Neo4j` 中看到我们构建的知识图谱了。如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503211705239.png" width=80%></div>